## Import mysql.connector, create connection to MySQL local server

In [1]:
import mysql.connector

In [2]:
mydb = mysql.connector.connect(auth_plugin='mysql_native_password', user="", password="")

## Create cursor, generate country_club database and selecting it

In [3]:
cursor = mydb.cursor(buffered=True)
cursor.execute('CREATE DATABASE country_club')
cursor.execute('USE country_club')

### Data was downloaded as an SQL script from https://sql.springboard.com/
### Script opened, read and closed
### Commands transfered to a list

In [4]:
file = open("country_club.sql", "r")
sql_script = file.read()
file.close()

In [5]:
sql_command_list = sql_script.split(';')

In [6]:
ommited_lines = []
len(sql_command_list)

33

### Executing the SQL commands to generate database. In some cases the program was skipping lines because were starting with invalid characters and this was fixed by finding the word INSERT in the line and staring the command from there

In [7]:
for line in sql_command_list:
    try:
        cursor.execute(line)
    except:
        try:
            cursor.execute(line[line.find("INSERT"):])
        except:
            ommited_lines.append(line)

In [8]:
ommited_lines

['\n']

### Showing tables imported

In [9]:
cursor.execute("SHOW TABLES")

In [10]:
for x in cursor:
    print(x)

('bookings',)
('facilities',)
('members',)


### Q1: Some of the facilities charge a fee to members, but some do not. Please list the names of the facilities that do.

In [11]:
cursor.execute("SELECT name FROM Facilities WHERE membercost = 0")
myresult = cursor.fetchall()
for x in myresult:
  print(x)

('Badminton Court',)
('Table Tennis',)
('Snooker Table',)
('Pool Table',)


### Q2: How many facilities do not charge a fee to members?

In [12]:
cursor.execute("SELECT COUNT( * ) AS facilities_no_charge FROM Facilities WHERE membercost = 0")
myresult = cursor.fetchall()
for x in myresult:
  print(x)

(4,)


### Q3: How can you produce a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost? Return the facid, facility name, member cost, and monthly maintenance of the facilities in question

In [13]:
cursor.execute("SELECT facid, name, membercost, monthlymaintenance \
                FROM Facilities \
                WHERE (membercost > 0) AND (membercost < 0.2 * monthlymaintenance)")
myresult = cursor.fetchall()
for x in myresult:
  print(x)

(0, 'Tennis Court 1', Decimal('5.0'), 200)
(1, 'Tennis Court 2', Decimal('5.0'), 200)
(4, 'Massage Room 1', Decimal('9.9'), 3000)
(5, 'Massage Room 2', Decimal('9.9'), 3000)
(6, 'Squash Court', Decimal('3.5'), 80)


### Q4: How can you retrieve the details of facilities with ID 1 and 5? Write the query without using the OR operator.

In [14]:
cursor.execute("SELECT * FROM Facilities WHERE facid = 1 \
                UNION \
                SELECT * FROM Facilities WHERE facid = 5")
myresult = cursor.fetchall()
for x in myresult:
  print(x)

(1, 'Tennis Court 2', Decimal('5.0'), Decimal('25.0'), 8000, 200)
(5, 'Massage Room 2', Decimal('9.9'), Decimal('80.0'), 4000, 3000)


### Q5: How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than 100? Return the name and monthly maintenance of the facilities in question.

In [15]:
cursor.execute("SELECT name, monthlymaintenance, \
                CASE WHEN monthlymaintenance >100 THEN  'expensive' ELSE  'cheap' END AS TYPE \
                FROM Facilities")
myresult = cursor.fetchall()
for x in myresult:
  print(x)

('Tennis Court 1', 200, 'expensive')
('Tennis Court 2', 200, 'expensive')
('Badminton Court', 50, 'cheap')
('Table Tennis', 10, 'cheap')
('Massage Room 1', 3000, 'expensive')
('Massage Room 2', 3000, 'expensive')
('Squash Court', 80, 'cheap')
('Snooker Table', 15, 'cheap')
('Pool Table', 15, 'cheap')


### Q6: You'd like to get the first and last name of the last member(s) who signed up. Do not use the LIMIT clause for your solution

In [16]:
cursor.execute("SELECT sub.firstname, sub.surname \
                FROM (SELECT firstname, surname, MAX(joindate) \
                      FROM Members WHERE surname !=  'GUEST') sub")
myresult = cursor.fetchall()
for x in myresult:
  print(x)

('Darren', 'Smith')


### Q7: How can you produce a list of all members who have used a tennis court? Include in your output the name of the court, and the name of the member formatted as   single column. Ensure no duplicate data, and order by the member name.

In [17]:
cursor.execute("SELECT DISTINCT CONCAT(members.firstname, ' ', members.surname) AS full_name \
                FROM country_club.Members members \
                JOIN (SELECT bookings.facid, bookings.memid FROM country_club.Bookings bookings \
                JOIN country_club.Facilities facilities \
                ON bookings.facid = facilities.facid AND facilities.name LIKE '%Tennis Court%') sub \
                ON members.memid = sub.memid AND members.firstname NOT LIKE 'GUEST%' ORDER BY 1")
myresult = cursor.fetchall()
for x in myresult:
  print(x)

('Anne Baker',)
('Burton Tracy',)
('Charles Owen',)
('Darren Smith',)
('David Farrell',)
('David Jones',)
('David Pinker',)
('Douglas Jones',)
('Erica Crumpet',)
('Florence Bader',)
('Gerald Butters',)
('Henrietta Rumney',)
('Jack Smith',)
('Janice Joplette',)
('Jemima Farrell',)
('Joan Coplin',)
('John Hunt',)
('Matthew Genting',)
('Millicent Purview',)
('Nancy Dare',)
('Ponder Stibbons',)
('Ramnaresh Sarwin',)
('Tim Boothe',)
('Tim Rownam',)
('Timothy Baker',)
('Tracy Smith',)


### Q8: How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than 30? Remember that guests have different costs to members (the listed costs are per half-hour 'slot'), and the guest user's ID is always 0. Include in your output the name of the facility, the name of the member formatted as a single column, and the cost. Order by descending cost, and do not use any subqueries.

In [18]:
cursor.execute("SELECT facilities.name, CONCAT(members.firstname,  ' ', members.surname) as full_name,\
                CASE WHEN bookings.memid = 0 THEN facilities.guestcost * bookings.slots \
                     ELSE facilities.membercost * slots END AS cost \
                FROM country_club.Bookings bookings \
                JOIN country_club.Members members \
                ON bookings.memid = members.memid AND DATE(bookings.starttime) = '2012-09-14' \
                JOIN country_club.Facilities facilities \
                ON bookings.facid = facilities.facid HAVING cost > 30 ORDER BY 3 DESC")
myresult = cursor.fetchall()
for x in myresult:
  print(x)

('Massage Room 2', 'GUEST GUEST', Decimal('320.0'))
('Massage Room 1', 'GUEST GUEST', Decimal('160.0'))
('Massage Room 1', 'GUEST GUEST', Decimal('160.0'))
('Massage Room 1', 'GUEST GUEST', Decimal('160.0'))
('Tennis Court 2', 'GUEST GUEST', Decimal('150.0'))
('Tennis Court 1', 'GUEST GUEST', Decimal('75.0'))
('Tennis Court 2', 'GUEST GUEST', Decimal('75.0'))
('Tennis Court 1', 'GUEST GUEST', Decimal('75.0'))
('Squash Court', 'GUEST GUEST', Decimal('70.0'))
('Massage Room 1', 'Jemima Farrell', Decimal('39.6'))
('Squash Court', 'GUEST GUEST', Decimal('35.0'))
('Squash Court', 'GUEST GUEST', Decimal('35.0'))


### Q9: This time, produce the same result as in Q8, but using a subquery.

In [19]:
cursor.execute("SELECT sub.fac_name, sub.full_name, sub.cost \
FROM (SELECT facilities.name AS fac_name, CONCAT(members.firstname,  ' ', members.surname) AS full_name, \
             CASE WHEN bookings.memid = 0 THEN facilities.guestcost * bookings.slots \
             ELSE facilities.membercost * slots END AS cost \
      FROM country_club.Bookings bookings \
      JOIN country_club.Members members ON bookings.memid = members.memid AND DATE(bookings.starttime) = '2012-09-14' \
      JOIN country_club.Facilities facilities ON bookings.facid = facilities.facid) sub \
      WHERE sub.cost > 30 \
      ORDER BY 3 DESC")
myresult = cursor.fetchall()
for x in myresult:
  print(x)

('Massage Room 2', 'GUEST GUEST', Decimal('320.0'))
('Massage Room 1', 'GUEST GUEST', Decimal('160.0'))
('Massage Room 1', 'GUEST GUEST', Decimal('160.0'))
('Massage Room 1', 'GUEST GUEST', Decimal('160.0'))
('Tennis Court 2', 'GUEST GUEST', Decimal('150.0'))
('Tennis Court 1', 'GUEST GUEST', Decimal('75.0'))
('Tennis Court 1', 'GUEST GUEST', Decimal('75.0'))
('Tennis Court 2', 'GUEST GUEST', Decimal('75.0'))
('Squash Court', 'GUEST GUEST', Decimal('70.0'))
('Massage Room 1', 'Jemima Farrell', Decimal('39.6'))
('Squash Court', 'GUEST GUEST', Decimal('35.0'))
('Squash Court', 'GUEST GUEST', Decimal('35.0'))


### Q10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members!

In [20]:
cursor.execute("SELECT facilities.name, \
                SUM(CASE WHEN bookings.memid = 0 THEN facilities.guestcost * bookings.slots \
                         ELSE facilities.membercost * bookings.slots END) AS total_revenue \
                FROM country_club.Facilities facilities \
                JOIN country_club.Bookings bookings \
                ON facilities.facid = bookings.facid \
                GROUP BY 1 HAVING total_revenue <1000 ORDER BY 2 ")
myresult = cursor.fetchall()
for x in myresult:
  print(x)

('Table Tennis', Decimal('180.0'))
('Snooker Table', Decimal('240.0'))
('Pool Table', Decimal('270.0'))
